In [2]:
# Import libraries
import os
import boto3
import sagemaker

from sagemaker import get_execution_role

In [3]:
# initialize preconfigurations
region = boto3.Session().region_name
role = get_execution_role()

In [4]:
print(region)

eu-central-1


In [5]:
print(role)

arn:aws:iam::933618615917:role/service-role/AmazonSageMakerServiceCatalogProductsUseRole


In [6]:
# Create S3 bucket

# This creates a default S3 bucket where we will upload our model.
bucket = 'sagemaker-iris-classifier'  #sagemaker.Session().default_bucket()

bucket_path = "https://s3-{}.amazonaws.com/{}".format(region, bucket)

print(bucket)
print(bucket_path)

sagemaker-iris-classifier
https://s3-eu-central-1.amazonaws.com/sagemaker-iris-classifier


In [ ]:
'''
Install xgboost as it is needed for loading the model from joblib dump file and test it before deployment.
Please note that the XGBoost version should be same as the version with which the model was trained locally in laptop.
'''

In [7]:
!conda install -y -c conda-forge xgboost==0.90

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: - 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/noarch::nbclient==0.5.2=pyhd8ed1ab_0
  - conda-forge/linux-64::matplotlib==3.3.4=py36h5fab9bb_0
  - conda-forge/noarch::qdarkstyle==2.8.1=pyhd8ed1ab_2
  - conda-forge/linux-64::scikit-image==0.16.2=py36hb3f55d8_0
  - conda-forge/noarch::python-language-server==0.36.2=pyhd8ed1ab_0
  - conda-forge/linux-64::widgetsnbextension==3.5.1=py36h5fab9bb_4
  - conda-forge/noarch::flake8==3.8.4=py_0
  - conda-forge/noarch::ipywidgets==7.6.3=pyhd3deb0d_0
  - conda-forge/noarch::typing-extensions==3.7.4.3=0
  - conda-forge/noarch::path.py==12.5.0=0
  - conda-forge/noarch::dask==2021.2.0=pyhd8ed1ab_0
  - conda-forge/noarch::nbformat==5.1.2=pyhd8ed1ab_1
  - conda-forge/linux-64::path==15.1.2=py36h5fab9bb_0
  - conda-forge/linux-64::nbconvert==6.0.7=py36

In [8]:
model_file_name = "DEMO-local-xgboost-classifier-model"

In [9]:
# Load the pre-trained model, and test it before deployment
import joblib
import xgboost

mymodel = joblib.load(model_file_name)

In [10]:
#import test file
import numpy as np


file_name = (
    "test_point_classifier_sample.csv"  # customize to your test file, will be 'mnist.single.test' if use data above
)

with open(file_name, "r") as f:
    mypayload = np.loadtxt(f, delimiter=",")
    
print(mypayload)    

mymodel.predict(mypayload)

[[5.4 3.  4.5 1.5]
 [5.6 3.  4.1 1.3]
 [6.3 2.8 5.1 1.5]
 [6.  3.  4.8 1.8]
 [5.1 3.3 1.7 0.5]]


array([1, 1, 1, 2, 0], dtype=int32)

In [11]:
# Create a tar.gz model file as this is the format required by Sagemaker for deployment.
mymodel._Booster.save_model(model_file_name)

In [12]:
!tar czvf model.tar.gz $model_file_name

DEMO-local-xgboost-classifier-model


In [13]:
# Upload the pre-trained model to S3

#### prefix in S3
prefix = "sagemaker/DEMO-xgboost-classifier-byo"

fObj = open("model.tar.gz", "rb")
key = os.path.join(prefix, model_file_name, "model.tar.gz")
print(key)
boto3.Session().resource("s3").Bucket(bucket).Object(key).upload_fileobj(fObj)

sagemaker/DEMO-xgboost-classifier-byo/DEMO-local-xgboost-classifier-model/model.tar.gz


In [ ]:
## Set up hosting for the model
'''
Import model into hosting
This involves creating a SageMaker model from the model file previously uploaded to S3.
'''

In [14]:
# Create a Sagemaker model
from sagemaker.amazon.amazon_estimator import get_image_uri

#### Get the built-in xgboost container image in Sagemaker to host our model
container = get_image_uri(boto3.Session().region_name, "xgboost", "0.90-1")

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [15]:
%%time
from time import gmtime, strftime

model_name = model_file_name + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

model_url = "https://s3-{}.amazonaws.com/{}/{}".format(region, bucket, key)

sm_client = boto3.client("sagemaker")

print(model_url)

primary_container = {
    "Image": container,
    "ModelDataUrl": model_url,
}

create_model_response2 = sm_client.create_model(
    ModelName=model_name, ExecutionRoleArn=role, PrimaryContainer=primary_container
)

print(create_model_response2["ModelArn"])

https://s3-eu-central-1.amazonaws.com/sagemaker-iris-classifier/sagemaker/DEMO-xgboost-classifier-byo/DEMO-local-xgboost-classifier-model/model.tar.gz
arn:aws:sagemaker:eu-central-1:933618615917:model/demo-local-xgboost-classifier-model2022-01-10-09-19-46
CPU times: user 80.6 ms, sys: 2.19 ms, total: 82.8 ms
Wall time: 461 ms


In [16]:
# Create endpoint configuration
'''
Create an endpoint configuration, that describes the distribution of traffic across the models, whether split, shadowed, or sampled in some way. 
In addition, the endpoint configuration describes the instance type required for model deployment.
'''
from time import gmtime, strftime

endpoint_config_name = "DEMO-XGBoost-Classifier-EndpointConfig-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

print(endpoint_config_name)

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "InstanceType": "ml.m4.xlarge",
            "InitialInstanceCount": 1,
            "InitialVariantWeight": 1,
            "ModelName": model_name,
            "VariantName": "AllTraffic",
        }
    ],
)

print("Endpoint Config Arn: " + create_endpoint_config_response["EndpointConfigArn"])

DEMO-XGBoost-Classifier-EndpointConfig-2022-01-10-09-19-55
Endpoint Config Arn: arn:aws:sagemaker:eu-central-1:933618615917:endpoint-config/demo-xgboost-classifier-endpointconfig-2022-01-10-09-19-55


In [18]:
# Create endpoint
'''
Lastly, you create the endpoint that serves up the model, through specifying the name and configuration defined above. 
The end result is an endpoint that can be validated and incorporated into production applications.
'''
import time
endpoint_name = "DEMO-XGBoost-Classifier-Endpoint-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_name)
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)
print(create_endpoint_response["EndpointArn"])

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

DEMO-XGBoost-Classifier-Endpoint-2022-01-10-09-20-25
arn:aws:sagemaker:eu-central-1:933618615917:endpoint/demo-xgboost-classifier-endpoint-2022-01-10-09-20-25
Status: Creating
Status: Creating
Status: Creating
Status: InService
Arn: arn:aws:sagemaker:eu-central-1:933618615917:endpoint/demo-xgboost-classifier-endpoint-2022-01-10-09-20-25
Status: InService


In [19]:
# Validate the model for use
'''
Now you can obtain the endpoint from the client library using the result from previous operations 
and generate classifications from the model using that endpoint.
'''
runtime_client = boto3.client("runtime.sagemaker")

In [34]:
%%time
import json


file_name = (
    "test_point_classifier_sample.csv"  # customize to your test file
)

with open(file_name, "r") as f:
    payload = f.read().strip()
    
    
print("Payload :\n")

print(payload)
print()

response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name, ContentType="text/csv", Body=payload
)

##print(response)

print("Results :\n")
print()

result = response["Body"].read().decode("utf-8")

# Unpack response
print("\nPredicted Class Probabilities: {}.".format(result))
classes = ['Setosa', 'Versicolor', 'Virginica']

Payload :

5.400000000000000355e+00,3.000000000000000000e+00,4.500000000000000000e+00,1.500000000000000000e+00
5.599999999999999645e+00,3.000000000000000000e+00,4.099999999999999645e+00,1.300000000000000044e+00
6.299999999999999822e+00,2.799999999999999822e+00,5.099999999999999645e+00,1.500000000000000000e+00
6.000000000000000000e+00,3.000000000000000000e+00,4.799999999999999822e+00,1.800000000000000044e+00
5.099999999999999645e+00,3.299999999999999822e+00,1.699999999999999956e+00,5.000000000000000000e-01

Results :



Predicted Class Probabilities: [0.056180261075496674, 0.887361466884613, 0.056458208709955215],[0.056180261075496674, 0.887361466884613, 0.056458208709955215],[0.08053058385848999, 0.8385403752326965, 0.08092901110649109],[0.12150664627552032, 0.27632802724838257, 0.6021653413772583],[0.8827555775642395, 0.0601295568048954, 0.05711490288376808].
CPU times: user 924 µs, sys: 4.27 ms, total: 5.19 ms
Wall time: 17 ms


In [ ]:
# Delete the Endpoint
sm_client.delete_endpoint(EndpointName=endpoint_name)
runtime_client.delete_endpoint(EndpointName=endpoint_name)